In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import requests

In [4]:
spark = SparkSession.builder\
    .appName("DataRequestApp")\
    .master("spark://spark-master:7077")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/30 15:12:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
import requests
response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=256da2d742d5a5979790e6833447e4b4').json()

# API Reference

[TMDB](https://developer.themoviedb.org/reference/intro/getting-started)

In [23]:
df = spark.createDataFrame(response['results'])

AttributeError: 'NoneType' object has no attribute 'sc'

In [7]:
df.show()

+-----+--------------------+--------------------+-------+-----------------+--------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+
|adult|       backdrop_path|           genre_ids|     id|original_language|      original_title|            overview|popularity|         poster_path|release_date|               title|video|vote_average|vote_count|
+-----+--------------------+--------------------+-------+-----------------+--------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+
|false|/euYIwmwkmz95mnXv...|        [28, 12, 18]| 558449|               en|        Gladiator II|Years after witne...|  7979.938|/2cxhvwyEwRlysAmR...|  2024-11-05|        Gladiator II|false|       6.781|      1622|
|false|/6qld2YxAO9gdEblo...|         [10749, 18]|1156593|               es|          Culpa tuya|The love between ...|  5541.871|/1sQA7lfcF9yUyoL

In [16]:
df_movies = df.select('title', 'release_date', 'overview', 'vote_average')

In [18]:
df_movies.show()

+--------------------+------------+--------------------+------------+
|               title|release_date|            overview|vote_average|
+--------------------+------------+--------------------+------------+
|        Gladiator II|  2024-11-05|Years after witne...|       6.781|
|          Your Fault|  2024-12-26|The love between ...|       7.737|
|             Red One|  2024-10-31|After Santa Claus...|       7.051|
|Sonic the Hedgehog 3|  2024-12-19|Sonic, Knuckles, ...|         7.8|
|Venom: The Last D...|  2024-10-22|Eddie and Venom a...|         6.8|
|             Moana 2|  2024-11-21|After receiving a...|       6.967|
|Mufasa: The Lion ...|  2024-12-18|Told in flashback...|         7.3|
|            My Fault|  2023-06-08|Noah must leave h...|       7.916|
|            Carry-On|  2024-12-05|An airport securi...|         7.0|
|        Dirty Angels|  2024-12-11|During the United...|         6.7|
|          Absolution|  2024-10-31|An aging ex-boxer...|         6.1|
|           Elevatio

In [21]:
df_movies.write.mode("overwrite").parquet("/home/jupyter/app/data/movies.parquet")

In [22]:
spark.stop()